In [1]:
import string
import numpy as np
import pandas as pd
import codecs
import joblib
import pymorphy3
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from annoy import AnnoyIndex

import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("./data/ProductsDataset.csv")
df.rename(columns={'descrirption': 'description', 'product_id' : 'answer'}, inplace=True)
data_market = df[['title', 'description', 'answer']]
#data_market = df[['title', 'answer']]
data_market['text'] = data_market['title'] + " " + data_market['description']
data_market.drop(['title', 'description'], axis=1, inplace=True)
#data_market.rename(columns={'title':'text'}, inplace=True)
data_market.dropna(inplace=True)

data_market.head()

,answer,text
0,58e3cfe6132ca50e053f5f82,"Юбка детская ORBY Новая, не носили ни разу. В ..."
1,5667531b2b7f8d127d838c34,"Ботильоны Новые,привезены из Чехии ,указан раз..."
2,59534826aaab284cba337e06,Брюки Размер 40-42. Брюки почти новые - не зна...
3,57de544096ad842e26de8027,"Продам детские шапки Продам шапки,кажда 200р.Р..."
4,5ad4d2626c86cb168d212022,"Блузка Темно-синяя, 42 размер,состояние отличн..."


In [3]:
morpher = pymorphy3.MorphAnalyzer()

stop_words = set(stopwords.words('russian'))
sw = set(stop_words)


def preprocess_txt(line):
    if isinstance(line, str):
        exclude = set(string.punctuation)
        line = line.replace(',', ', ')
        line = line.replace('.', '. ')
        line = line.replace('не ', 'не')
        spls = "".join(i for i in line if i not in exclude).strip().split()
        spls = [morpher.parse(i.lower())[
            0].normal_form for i in spls if i.lower() not in sw and i != ""]
        
        spls = ' '.join(spls)
        
        return spls
    else:
        return str(line)


data_market['text'] = data_market['text'].apply(preprocess_txt)
data_market['class'] = 1

In [4]:
data_market.head()

,answer,text,class
0,58e3cfe6132ca50e053f5f82,юбка детский orby новый неносить раз реал крас...,1
1,5667531b2b7f8d127d838c34,ботильон новый привезти чехия указать размер 4...,1
2,59534826aaab284cba337e06,брюки размер 4042 брюки новый незнать мерило п...,1
3,57de544096ad842e26de8027,продать детский шапка продать шапка кажда 200р...,1
4,5ad4d2626c86cb168d212022,блузка темносиний 42 размер состояние отличный...,1


In [5]:
# Read the txt file into a dataframe, skipping bad lines
data_talker = pd.read_csv(
    './data/prepared_answers.txt', sep='\t',
    header=None, names=['text', 'answer'], 
    on_bad_lines="skip"
)

data_talker.dropna(inplace=True)
data_talker['class'] = 0

data_talker = data_talker.iloc[:len(data_market)]

data_talker['text'] = data_talker['text'].apply(preprocess_txt)

data_talker.head()

,text,answer,class
1,парень относиться цветной линза девушка зелёны...,меня вобще прикалывает эта тема :).,0
2,делать сегодня найти 2 миллион рубль,"Если это ""счастье "" действительно на вас свали...",0
3,эбу двенашка называться итэлма эбу,ЭБУ — электронный блок управления двигателем а...,0
4,академия вампир сколько даный момент часть кни...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...",0
5,защититься энергетический вампир,Защита мыслью. <br>Каждый человек должен в отн...,0


In [6]:
cdf = pd.concat([data_market, data_talker], axis=0)

display(cdf['class'].value_counts())
display(cdf.head(), cdf.tail())

class
1    33534
0    33534
Name: count, dtype: int64

,answer,text,class
0,58e3cfe6132ca50e053f5f82,юбка детский orby новый неносить раз реал крас...,1
1,5667531b2b7f8d127d838c34,ботильон новый привезти чехия указать размер 4...,1
2,59534826aaab284cba337e06,брюки размер 4042 брюки новый незнать мерило п...,1
3,57de544096ad842e26de8027,продать детский шапка продать шапка кажда 200р...,1
4,5ad4d2626c86cb168d212022,блузка темносиний 42 размер состояние отличный...,1


,answer,text,class
33530,<p>Никола́й Дми́триевич Серге́ев (22 сентября ...,сергеев николай дмитриевич,0
33531,Считают.... что это не модно.... Глубоко заблу...,романтик просто почему романтик мало просто не...,0
33532,Ноты.....,самый музыкальный термин,0
33533,Герпес - это вирус. Очень у многих людей вирус...,герпес это такой лечить себя ухаживать интерне...,0
33534,"<p> 45 орехов если первая белка -х, то вторая ...",первый белок найти несколько орех это 13 часть...,0


In [7]:
X = cdf['text']
y = cdf['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)



In [8]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3), max_features=100000).fit(
    X_train.values)

xtrain_tfidf = tfidf_vec.transform(X_train)
xtest_tfidf = tfidf_vec.transform(X_test)


In [9]:
model_clf = LogisticRegression(random_state=42)
model_clf.fit(xtrain_tfidf, y_train)
predictions = model_clf.predict(xtest_tfidf)

print(accuracy_score(y_test, predictions))

0.9847174593708066


In [10]:
# Сохранение TfidfVectorizer в файл
joblib.dump(tfidf_vec, './data/tfidf_vectorizer.pkl')
tfidf_vectorizer = joblib.load('./data/tfidf_vectorizer.pkl')

In [11]:
new_data = 'крокодилья шапка'
question = tfidf_vec.transform(pd.Series(new_data))
answer = model_clf.predict(question)
print(answer)

[1]


---

In [12]:
# Функция классификации
def responce_classification(text=''):
    question = tfidf_vectorizer.transform(pd.Series(text))
    label = model_clf.predict(question)
    return label[0]

In [13]:
data_market_results = data_market.drop(['class'], axis=1)
data_market_results

,answer,text
0,58e3cfe6132ca50e053f5f82,юбка детский orby новый неносить раз реал крас...
1,5667531b2b7f8d127d838c34,ботильон новый привезти чехия указать размер 4...
2,59534826aaab284cba337e06,брюки размер 4042 брюки новый незнать мерило п...
3,57de544096ad842e26de8027,продать детский шапка продать шапка кажда 200р...
4,5ad4d2626c86cb168d212022,блузка темносиний 42 размер состояние отличный...
...,...,...
35542,5ac078a7938000715c0e2c38,новый болеро soliver привезти европа неподойти...
35543,5b5f181c62e1c6616a7f6472,юбка юбка белый турция фирма adl
35544,5bd6c8b29e94ba033d31f8d0,новый твидовый пиджак новый бирка пиджак разме...
35545,5bd6c8bc074b3e1c056f69b2,женский зимний куртка женский зимний спортивны...


In [14]:
# Создание AnnoyIndex для маркета
market_tfidf = tfidf_vectorizer.transform(data_market_results['text'])
vector_size = market_tfidf.shape[1]  # Размерность векторов
index_market = AnnoyIndex(vector_size, 'angular')  # 'angular' для использования косинусной меры

# Добавление векторов из обучающего набора в AnnoyIndex
for i, value in data_market_results['text'].items():
    vector = tfidf_vectorizer.transform([value]).toarray()[0]  # Преобразуйте разреженную матрицу TF-IDF в массив
    index_market.add_item(i, vector)

# Построение индекса
index_market.build(10)  # Замените n_trees на желаемое количество деревьев
index_market.save('./data/market_idx.ann')


True

In [15]:
talker = data_talker.drop(['class'], axis=1)
talker

,text,answer
1,парень относиться цветной линза девушка зелёны...,меня вобще прикалывает эта тема :).
2,делать сегодня найти 2 миллион рубль,"Если это ""счастье "" действительно на вас свали..."
3,эбу двенашка называться итэлма эбу,ЭБУ — электронный блок управления двигателем а...
4,академия вампир сколько даный момент часть кни...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."
5,защититься энергетический вампир,Защита мыслью. <br>Каждый человек должен в отн...
...,...,...
33530,сергеев николай дмитриевич,<p>Никола́й Дми́триевич Серге́ев (22 сентября ...
33531,романтик просто почему романтик мало просто не...,Считают.... что это не модно.... Глубоко заблу...
33532,самый музыкальный термин,Ноты.....
33533,герпес это такой лечить себя ухаживать интерне...,Герпес - это вирус. Очень у многих людей вирус...


In [16]:
# Создание AnnoyIndex для маркета
talker_tfidf = tfidf_vectorizer.transform(talker['text'])
vector_size = talker_tfidf.shape[1]  # Размерность векторов
index_talker = AnnoyIndex(vector_size, 'angular')  # 'angular' для использования косинусной меры

# Добавление векторов из обучающего набора в AnnoyIndex
for i, value in talker['text'].items():
    vector = tfidf_vectorizer.transform([value]).toarray()[0]  # Преобразуйте разреженную матрицу TF-IDF в массив
    index_talker.add_item(i, vector)

# Построение индекса
index_talker.build(10)  # Замените n_trees на желаемое количество деревьев
index_talker.save('./data/talker_idx.ann')


True

In [20]:
new_data = 'orby'
question = tfidf_vectorizer.transform(pd.Series(new_data))

# Поиск ближайших соседей для вопроса
nearest_neighbors = index_market.get_nns_by_vector(question.toarray()[0], 1, search_k=2)

# Вывод наиболее релевантного ответа
answer = data_market_results['answer'].iloc[nearest_neighbors]

display(data_market_results[data_market_results['answer'] == answer.values[0]])
print(answer)

,answer,text
14523,5a92ef916c86cb68a245e5ea,сапог кожа осенний натуральный кожа италия 37 рр


14523    5a92ef916c86cb68a245e5ea
Name: answer, dtype: object


In [18]:
question = tfidf_vectorizer.transform(pd.Series(new_data))

# Поиск ближайших соседей для вопроса
nearest_neighbors = index_talker.get_nns_by_vector(question.toarray()[0], 1, search_k=1)

# Вывод наиболее релевантного ответа
answer = data_talker['answer'].iloc[nearest_neighbors]
print(answer)

2    Если это "счастье " действительно на вас свали...
Name: answer, dtype: object


----

In [19]:
# Создание AnnoyIndex
vector_size = xtrain_tfidf.shape[1]  # Размерность векторов
index_ann = AnnoyIndex(vector_size, 'angular')  # 'angular' для использования косинусной меры

# Добавление векторов из обучающего набора в AnnoyIndex
for i, value in cdf['text'].items():
    vector = tfidf_vec.transform([value]).toarray()[0]  # Преобразуйте разреженную матрицу TF-IDF в массив
    index_ann.add_item(i, vector)

# Построение индекса
index_ann.build(10)  # Замените n_trees на желаемое количество деревьев


KeyboardInterrupt: 

In [ ]:
new_data = 'Юбка детская ORBY '
question = tfidf_vec.transform(pd.Series(new_data))

# Поиск ближайших соседей для вопроса
nearest_neighbors = index_ann.get_nns_by_vector(question.toarray()[0], 1, search_k=1)

# Вывод наиболее релевантного ответа
answer = cdf['answer'].iloc[nearest_neighbors]
print(answer)

2010    Оч точно подмечено!! ! +100000. 
Name: answer, dtype: object


In [ ]:
cdf[cdf['answer'] ==answer.values[0]]

,answer,text,class
2010,Оч точно подмечено!! ! +100000.,заметить комментарий смешной сам вопрос,0
